# Echo State Network against Time Series

In [ ]:
import time

import torch
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np

from qbraid_algorithms import EchoStateNetwork, EchoStateReservoir

Format Time Series data to work with the ESN

In [ ]:
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        seq_x, seq_y = data[i:i+n_steps], data[i+n_steps]
        X.append(seq_x)
        y.append(seq_y)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32).view(-1,1, 1)

n_steps = 10

t = np.linspace(0, 2 * np.pi * 10, 1000)
data = np.sin(t)
X, y = create_sequences(data, n_steps)
trainset = TensorDataset(X, y)
trainloader = DataLoader(trainset, shuffle=True)

Initialize ESN, optimizer, and loss criterion

In [ ]:
input_size = 10
output_size = 1
hyperparams = {
    "hidden_size": 2500,
    "sparsity": 0.9,
    "spectral_radius": 0.99,
    "a": 0.6,
    "leak": 1.0,
}

reservoir = EchoStateReservoir(input_size, **hyperparams)
esn = EchoStateNetwork(reservoir, output_size).float()

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(esn.parameters(), lr=0.00001)
len_data = len(trainset)
nsamples = 1000
nepochs = 200

Train the network

In [ ]:
loss_values = []
start = time.time()
for epoch in range(nepochs):
    total_loss = 0.0
    for i, dat in enumerate(trainset, 0):
        seq = dat[0]
        target = dat[1] 

        optimizer.zero_grad()

        output = esn(seq)[0]  
        loss = criterion(output, target) 

        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    average_loss = total_loss / nsamples
    loss_values.append(average_loss)
    print(f"Epoch {epoch+1}, Loss: {average_loss}")

end = time.time()
print("Training complete")
print("num samples = " + str(nsamples))
print("num epochs = " + str(nepochs))
print("total time = " + str(end - start) + " sec")


Plot the loss

In [ ]:
fig, ax = plt.subplots(dpi=100)
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")
ax.plot(range(1, nepochs + 1), loss_values, color="red")
plt.show()

Plot against desired data, giving R^2 as well

In [ ]:
def test():
    t_test = np.linspace(2 * np.pi * 10, 2 * np.pi * 12, 400)  # continuing from where training data stopped
    data_test = np.sin(t_test)
    X_test, y_test = create_sequences(data_test, n_steps)
    testset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(testset, shuffle=True)
    esn.eval()
    predictions = []
    with torch.no_grad():
        for inputs, targets in test_loader.dataset:
            output = esn(inputs)[0]
            predictions.append(output)

    return predictions


predictions = test()
t_test = np.linspace(2 * np.pi * 10, 2 * np.pi * 12, 400)
plt.plot(torch.cat(predictions).numpy(), label="Predictions")
plt.plot([np.sin(y) for y in t_test], label="True values")
plt.legend()

y_true = np.sin(t_test)
y_pred = torch.cat(predictions).numpy().flatten()
y_true_mean = np.mean(y_true)
ss_tot = np.sum((y_true - y_true_mean) ** 2)
ss_res = np.sum((y_true[:390] - y_pred) ** 2)
r2 = 1 - ss_res / ss_tot
print("R^2: ", r2)
